---

**Advanced Lane Finding Project**

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]: ./examples/undistort_output.png "Undistorted"
[image2]: ./examples/test1.jpg "Road Transformed"
[image3]: ./examples/binary.png "Binary Example"
[image4]: ./examples/warped.png "Warp Example"
[image5]: ./examples/lines.png "Fit Visual"
[image6]: ./examples/result.png "Output"
[video1]: ./project_video_ouput.mp4 "Video"

### [Rubric](https://review.udacity.com/#!/rubrics/571/view) Points

#### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---

### Camera Calibration

The code for this step is contained in the first code cell of the IPython notebook located in "P4.ipynb".  

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.  

I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  I applied this distortion correction to the test image using the `cv2.undistort()` function and obtained this result: 

![alt text][image1]

### Pipeline (single images)

#### 1. Distortion-corrected image.

To demonstrate this step, I will describe how I apply the distortion correction to one of the test images like this one:
![alt text][image2]

#### 2. Binary image

I used a combination of V channel in HSV color space, sobel filter and gradient thresholds to generate a binary image in function `binary_images()`.  Here's an example of my output for this step.

![alt text][image3]

#### 3. Perspective transform

The code for my perspective transform includes a function called `perspective_transform()`. The `perspective_transform()` function takes as inputs an image (`img`), as well as source (`src`) and destination (`dst`) points.  I chose the hardcode the source and destination points in the following manner:

```python
src = np.float32([[200,700],[1250,700],[580,460],[750,460]])
dst = np.float32([[200,700],[1280,700],[200,0],[1280,0]])
```

This resulted in the following source and destination points:

| Source        | Destination   | 
|:-------------:|:-------------:| 
| 580, 460      | 200, 0        | 
| 200, 720      | 200, 700      |
| 1250, 720     | 1280, 700      |
| 750, 460      | 1280, 0        |

I verified that my perspective transform was working as expected by drawing the `src` and `dst` points onto a test image and its warped counterpart to verify that the lines appear parallel in the warped image.

![alt text][image4]

#### 4. Identify lane-line pixels and fit their positions with a polynomial

Then I did some other stuff and fit my lane lines with a 2nd order polynomial kinda like this:

![alt text][image5]

#### 5. Calculate the radius of curvature of the lane and the position of the vehicle with respect to center.

I did this in function `curvature_position()`.

#### 6. Result plotted back down onto the road such that the lane area is identified clearly.

I implemented this step in function `warp_back()`.  Here is an example of my result on a test image:

![alt text][image6]

---

### Pipeline (video)

Here's a [link to my video result](./project_video_ouput.mp4)

---

### Discussion

Here I'll talk about the approach I took,I use V channel of images in HSV space to make the yellow lines and white lines clearer even if the shadow of the trees lie on them. Then I use sobel filter to detect the lines. In order to detect them clearly and stablely, I just search in small area in histogram of the lines which is [150,300] on left lines and [1100,1250] on right lines. Besides when fitting the lines, I use the previous 20 paramters of the lines with the current parameters to get an average parmters of the lines. These techniques will ensure the lines detected accurately and stablely even if there are lots of noise in transformed images. However this technique is just suitble for this project video, because the V channel cannot ensure to detect the lines when the color of the road changes, and 20 previous images may not work well when facing different curvature of the lines. Further more when the lines are not clear or there are lots of wheel traces on the road, the technique cannot ensure the correctness.

In order to make it more robust, I can use color space to extract yellow lines and white lines seperately after removing the shadow on the lines. 